In [12]:
import pandas as pd
import numpy as np

from sklearn import preprocessing

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import linear_kernel

import re
import string


# Data Loading and Preprocessing

In [2]:
df = pd.read_csv('food_items.csv')
df.head()

,Food_ID,Name,C_Type,Veg_Non,Describe
0,1,summer squash salad,Healthy Food,veg,"white balsamic vinegar, lemon juice, lemon rin..."
1,2,chicken minced salad,Healthy Food,non-veg,"olive oil, chicken mince, garlic (minced), oni..."
2,3,sweet chilli almonds,Snack,veg,"almonds whole, egg white, curry leaves, salt, ..."
3,4,tricolour salad,Healthy Food,veg,"vinegar, honey/sugar, soy sauce, salt, garlic ..."
4,5,christmas cake,Dessert,veg,"christmas dry fruits (pre-soaked), orange zest..."


# Text Cleaning Function

In [3]:
def text_cleaning(text):
    text = "".join([char for char in text if char not in string.punctuation])
    return text

In [4]:
df['Describe'] = df['Describe'].apply(text_cleaning)

In [5]:
df.duplicated().sum()

0

In [6]:
df.isnull().sum()

Food_ID     0
Name        0
C_Type      0
Veg_Non     0
Describe    0
dtype: int64

In [7]:
df.describe()

,Food_ID
count,400.000000
mean,200.500000
std,115.614301
min,1.000000
25%,100.750000
50%,200.500000
75%,300.250000
max,400.000000


# Collaborative Filtering

In [8]:
rating = pd.read_csv('ratings.csv')
food_rating = rating.groupby(by='Food_ID').count()['Rating'].reset_index().rename(columns={'Rating':'Rating_count'})
user_rating = rating.groupby(by='User_ID').count()['Rating'].reset_index().rename(columns={'Rating':'Rating_count'})
rating_matrix = rating.pivot_table(index='Food_ID', columns='User_ID', values='Rating').fillna(0)
csr_rating_matrix = csr_matrix(rating_matrix.values)

In [9]:
recommender = NearestNeighbors(metric='cosine')
recommender.fit(csr_rating_matrix)

NearestNeighbors(metric='cosine')

Content-Based Filtering

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Describe'])

content_similarity = linear_kernel(tfidf_matrix, tfidf_matrix)

# Recommender Function

for Collaborative Filtering

In [18]:
def Collaborative_Recommender(title):
    user = df[df['Name'] == title]
    user_index = np.where(rating_matrix.index == int(user['Food_ID']))[0][0]
    user_ratings = rating_matrix.iloc[user_index]

    reshaped = user_ratings.values.reshape(1, -1)
    distances, indices = recommender.kneighbors(reshaped, n_neighbors=16)

    nearest_neighbors_indices = rating_matrix.iloc[indices[0]].index[1:]
    nearest_neighbors = pd.DataFrame({'Food_ID': nearest_neighbors_indices})

    result = pd.merge(nearest_neighbors, df, on='Food_ID', how='left')

    return result.head()

for Content Based Filtering

In [16]:
def Content_Recommender(title):
    content_index = df[df['Name'] == title].index[0]
    content_scores = list(enumerate(content_similarity[content_index]))
    content_scores = sorted(content_scores, key=lambda x: x[1], reverse=True)
    content_scores = content_scores[1:16]
    content_indices = [i[0] for i in content_scores]
    content_recommendations = df.iloc[content_indices][['Food_ID', 'Name', 'Describe']]

    return content_recommendations.head()

# Example Usage

In [19]:
collaborative_recommended_items = Collaborative_Recommender('tricolour salad')
content_recommended_items = Content_Recommender('tricolour salad')

In [20]:
print(collaborative_recommended_items)

   Food_ID                               Name    C_Type  Veg_Non  \
0    126.0            andhra crab meat masala    Indian  non-veg   
1     75.0                    detox haldi tea  Beverage      veg   
2    100.0                spicy chicken curry    Indian  non-veg   
3    259.0         ragi coconut ladoo (laddu)   Dessert      veg   
4     51.0  christmas chocolate fudge cookies   Dessert      veg   

                                            Describe  
0  processed crab meat refined oil curry leaves g...  
1              haldi ginger black pepper honey water  
2  oil ghee onion paste garlic paste ginger paste...  
3  finger millet flour ragi jaggery peanuts cocon...  
4  unsalted butter brown sugar chocolate chocolat...  


In [21]:
print(content_recommended_items)

     Food_ID                     Name  \
103      104           chilli chicken   
1          2     chicken minced salad   
27        28  vegetable som tam salad   
282      283        veg hakka noodles   
166      167           veg fried rice   

                                              Describe  
103  boneless chicken salt cornflour black pepper e...  
1    olive oil chicken mince garlic minced onion sa...  
27   raw papaya carrot french bean diamond cherry t...  
282  noodles salt oil garlic paste ginger paste bea...  
166  oil gingergarlic paste spring onion carrot cab...  
